Подготавливаем HTML для парсинга

In [1]:
from bs4 import BeautifulSoup
from requests import get

base_url = 'https://sensors.mwlabs.ru'
bs = BeautifulSoup(get(base_url).text)
bs.prettify()

'<html>\n <body>\n  <p>\n   04.12.2021 13:38:31 !!!\n  </p>\n  <table border="">\n   hellohello1\n   <tr style="background:navy;color:white;">\n    <td colspan="3">\n     -\n    </td>\n   </tr>\n   <tr>\n    <td>\n     <a href="/view/616412704B56" target="_blank">\n      616412704B56\n     </a>\n    </td>\n    <td style=";background-color:red;color:white;">\n     <p style="font-size:18pt;text-align:right;padding:0px;margin:0px;">\n      -1.69\n     </p>\n     <p style="font-size:8pt;text-align:right;padding:0px;margin:0px;">\n      07.04.2020 21:44\n     </p>\n    </td>\n    <td>\n     <a href="/edit/616412704B56">\n      edit\n     </a>\n    </td>\n   </tr>\n   <tr>\n    <td>\n     <a href="/view/616412731311" target="_blank">\n      616412731311\n     </a>\n    </td>\n    <td style=";background-color:red;color:white;">\n     <p style="font-size:18pt;text-align:right;padding:0px;margin:0px;">\n      -3.87\n     </p>\n     <p style="font-size:8pt;text-align:right;padding:0px;margin:0px

Парсим ссылки на данные, по которым идёт сбор до сих пор, попутно формируя имена для рядов исходя из аудитории и расположения датчика в ней

In [2]:
from itertools import takewhile

sensors_urls = {
    '_'.join((tr_place.td.a.text, tr_class.td.text.replace('-', '_'))): tr_place.td.a.get('href')
    for tr_class in bs.find_all('tr')
    for tr_place in (
        sibling
        for sibling in takewhile(
            lambda sibling: not bool(sibling.get('style')),
            tr_class.find_next_siblings('tr')
        )
        if all(not bool(td.get('style')) for td in sibling.find_all('td'))
    )
    if bool(tr_class.get('style'))
}
sensors_urls

{'bat_210': '/view/ABE679970903',
 'bat_316': '/view/0A7779970903',
 'bat_412_a': '/view/7D0679971403',
 'bat_420': '/view/53C779971203',
 'wall_210': '/view/43E079971203',
 'wall_316': '/view/94CC79971203',
 'wall_412_a': '/view/A2BF79971203',
 'wall_420': '/view/6AE379971203'}


Загружаем данные по полученным ссылкам и сохраняем их на диск

In [3]:
from csv import writer
from dateutil.parser import parse
from pathlib import Path
from urllib.parse import urljoin

temperatures_directory = Path.cwd() / '..' / 'data' / 'temperatures' / parse(bs.text[:19]).isoformat() / '1_raw'

temperatures_directory.mkdir(parents=True, exist_ok=True)

for sensor, url in sensors_urls.items():
    with open(temperatures_directory / f'{sensor}.csv', 'w') as file:
        writer(file).writerows(
            (measurement[:19], measurement[20:])
            for measurement in get(urljoin(base_url, url)).text.split('\r\n')[:-1]
        )